In [1]:
import paddle
from ernie.modeling_paddleocr_vl import PaddleOCRVLForConditionalGeneration
from ernie.tokenizer import Ernie4_5_Tokenizer
from data_processor.image_preprocessor.image_preprocessor_siglip import SiglipImageProcessor
from paddleformers.peft import LoRAModel
from paddleformers.peft import LoRAConfig, LoRAModel, PrefixConfig, PrefixModelForCausalLM
from paddleformers.transformers import AutoModelForCausalLM, AutoTokenizer
from paddleformers.peft import LoRAAutoModel
BASE = "PaddlePaddle/PaddleOCR-VL"                     # 학습 베이스와 동일
LORA = "/home/mango/ERNIE/PaddleOCR-VL-SFT-OCRPUBLIC/checkpoint-250000"     # LoRA 출력 디렉터리(ERNIE 포맷)

# 1) 베이스 로드(HF->Paddle 변환 필요 시)
base_model = PaddleOCRVLForConditionalGeneration.from_pretrained(BASE, convert_from_hf=True)
tokenizer = Ernie4_5_Tokenizer.from_pretrained(LORA, padding_side="right", model_max_length=4096)
image_preprocess = SiglipImageProcessor.from_pretrained(LORA)

#lora_config = LoRAConfig.from_pretrained(LORA)
#dtype = lora_config.dtype
#lora_config.merge_weights = True
# 2) LoRA 어댑터 적용

lora_model = LoRAModel.from_pretrained(base_model, lora_path=LORA)
lora_model.merge()
lora_model.restore_original_model()
if hasattr(lora_model, "model"):
    # LoRAModel이 감싸고 있는 실제 모델 객체 접근
    merged_base_model = lora_model.model
    #print(merged_base_model)
    merged_base_model.save_pretrained(
    LORA,
    safe_serialization=True,   # HF safetensors로 저장
)
#lora_model.model.save_pretrained(LORA)

/root/anaconda3/envs/deepseek-ocr/lib/python3.12/site-packages/paddle/utils/cpp_extension/extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
W1229 13:58:17.316450 14527 gpu_resources.cc:114] Please NOTE: device: 0, GPU Compute Capability: 8.6, Driver API Version: 12.6, Runtime API Version: 11.8
[2025-12-29 13:58:22,033] [    INFO] - Using download source: huggingface
[2025-12-29 13:58:22,034] [    INFO] - Loading configuration file PaddlePaddle/PaddleOCR-VL/config.json
[2025-12-29 13:58:22,036] [    INFO] - Loading weights file PaddlePaddle/PaddleOCR-VL/model.safetensors
[2025-12-29 13:58:27,043] [    INFO] - Loaded weights file from disk, setting weights to model.
[2025-12-29 13:58:27,277] [    INFO] - use GQA - num_heads: 16- num_key_value_heads: 2
[2025-12-29 13:58:27,282] [    INF

In [ ]:
import os
try:
    from safetensors import safe_open
    from safetensors.torch import save_file
    import torch 
except ImportError:
    print("❌ 'pip install safetensors torch' 필요")
    exit()

# =========================================================
# [설정] 문제가 있는 LoRA 모델 경로
# =========================================================
TARGET_FILE = "/home/mango/ERNIE/PaddleOCR-VL-SFT-OCRPUBLIC/checkpoint-250000/model.safetensors"
# =========================================================

def fix_transposed_weights():
    print(f"📂 파일 로드 중: {TARGET_FILE}")
    
    if not os.path.exists(TARGET_FILE):
        print("❌ 파일 없음")
        return

    tensors_to_save = {}
    fixed_count = 0
    
    with safe_open(TARGET_FILE, framework="pt", device="cpu") as f:
        for key in f.keys():
            tensor = f.get_tensor(key)
            
            # 2D 텐서(Linear Weight)인 경우 전치 대상인지 확인
            if len(tensor.shape) == 2:
                # Transpose 대상 키워드 필터링
                if "proj.weight" in key or "fc1.weight" in key or "fc2.weight" in key or "lm_head.weight" in key or "linear" in key:
                    print(f"🔧 Transposing: {key} {tensor.shape} -> {tensor.t().shape}")
                    
                    # [핵심 수정] .t() 뒤에 .contiguous() 추가
                    tensor = tensor.t().contiguous() 
                    
                    fixed_count += 1
            
            tensors_to_save[key] = tensor

    # 3. 저장
    if fixed_count > 0:
        print(f"\n💾 총 {fixed_count}개의 텐서를 뒤집어 저장합니다...")
        output_path = TARGET_FILE.replace(".safetensors", "_fixed.safetensors")
        
        # 메타데이터 없이 저장 (순수 텐서 데이터)
        save_file(tensors_to_save, output_path)
        
        print(f"✅ 저장 완료: {output_path}")
        print("👉 이 파일의 이름을 'model.safetensors'로 변경해서 사용하세요!")
    else:
        print("⚠️ 수정할 텐서를 찾지 못했습니다.")

if __name__ == "__main__":
    fix_transposed_weights()

📂 파일 로드 중: /home/mango/ERNIE/PaddleOCR-VL-SFT-OCRPUBLIC/checkpoint-250000/model.safetensors
🔧 Transposing: lm_head.weight torch.Size([1024, 103424]) -> torch.Size([103424, 1024])
🔧 Transposing: mlp_AR.linear_1.weight torch.Size([4608, 4608]) -> torch.Size([4608, 4608])
🔧 Transposing: mlp_AR.linear_2.weight torch.Size([4608, 1024]) -> torch.Size([1024, 4608])
🔧 Transposing: model.layers.0.mlp.down_proj.weight torch.Size([3072, 1024]) -> torch.Size([1024, 3072])
🔧 Transposing: model.layers.0.mlp.gate_proj.weight torch.Size([1024, 3072]) -> torch.Size([3072, 1024])
🔧 Transposing: model.layers.0.mlp.up_proj.weight torch.Size([1024, 3072]) -> torch.Size([3072, 1024])
🔧 Transposing: model.layers.0.self_attn.k_proj.weight torch.Size([1024, 256]) -> torch.Size([256, 1024])
🔧 Transposing: model.layers.0.self_attn.o_proj.weight torch.Size([2048, 1024]) -> torch.Size([1024, 2048])
🔧 Transposing: model.layers.0.self_attn.q_proj.weight torch.Size([1024, 2048]) -> torch.Size([2048, 1024])
🔧 Transpos

: 